# Read the data

In [19]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = (40, 10)
sns.set_style("whitegrid")

In [20]:
threshold = 0.8
year_cutoff = 2000

In [21]:
def read_module(path, cutoff, threshold):
    # create a row filter
    skiprows = pd.read_csv(path, usecols=['syear'])['syear'] < cutoff
    skiprows = skiprows[skiprows].index.to_numpy() + 1
    # Read the data
    columns = pd.read_csv(path, nrows=1).columns
    batches = [columns[i:i+400] for i in range(0, len(columns), 400)]
    usecols = []
    for batch in tqdm(batches):
        chunk = pd.read_csv(path, usecols=batch, skiprows=skiprows)
        chunk = chunk.select_dtypes(include='number')
        # fitler nan values
        n = chunk.shape[0]
        for column in chunk.columns:
            nans = np.sum((chunk[column] < 0) | (chunk[column].isna()))
            usecols.append((column, 1 - nans/n))
    usecols = [x for x, y in usecols if y > threshold]
    return pd.read_csv(path, usecols=usecols, skiprows=skiprows)

def read_variables(path_values, path_variables):
    variables = pd.DataFrame()
    values = pd.DataFrame()
    for pval, pvar in zip(path_values, path_variables):
        var = pd.read_csv(
            pvar,
            usecols = ['variable', 'label_de']
        )
        var.rename(columns={'label_de':'variable_label'}, inplace=True)
        val = pd.read_csv(
            pval,
            usecols=['variable', 'value', 'label_de']
        )
        val.rename(columns={'label_de':'value_label'}, inplace=True)
        variables = pd.concat([variables, var], axis=0, ignore_index=True)
        values = pd.concat([values, val], axis=0, ignore_index=True)
    return variables, values


In [22]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

pl_path = './data/soep/CSV/pl.csv'
pl = read_module(pl_path, year_cutoff, threshold)
# calculate age
pl['age'] = pl['syear']
pl['age'] -= pl['ple0010_h']
# calculate date
time = pl[['syear', 'pmonin', 'ptagin']].rename(columns={'syear':'year', 'pmonin':'month', 'ptagin':'day'})
pl['time'] = pd.to_datetime(time, errors='coerce')
pl.to_parquet("./data/pl.parquet.gzip", index=False)

hl_path = './data/soep/CSV/hl.csv'
hl = read_module(hl_path, year_cutoff, threshold)
hl.to_parquet("./data/hl.parquet.gzip", index=False)

health_path = './data/soep/CSV/health.csv'
health = read_module(health_path, year_cutoff, 0.4)
health.to_parquet("./data/health.parquet.gzip", index=False)

regionl_path = './data/soep/CSV/regionl.csv'
regionl = read_module(regionl_path, year_cutoff, 0.8)
regionl.to_parquet("./data/regionl.parquet.gzip", index=False)

print(f"Selected {health.shape[1]} columns from health.")
print(f"Selected {pl.shape[1]} columns from pl.")
print(f"Selected {hl.shape[1]} columns from hl.")
print(f"Selected {regionl.shape[1]} columns from regionl.")

100%|██████████| 1/1 [00:00<00:00,  2.20it/s]


Selected 19 columns from health.
Selected 60 columns from pl.
Selected 36 columns from hl.
Selected 5 columns from regionl.


In [23]:

# Read variable labels
path_values = [
    './data/soep/CSV/pl_values.csv',
    './data/soep/CSV/hl_values.csv',
    './data/soep/CSV/health_values.csv',
    './data/soep/CSV/regionl_values.csv'
]
path_variables = [
    './data/soep/CSV/pl_variables.csv',
    './data/soep/CSV/hl_variables.csv',
    './data/soep/CSV/health_variables.csv',
    './data/soep/CSV/regionl_variables.csv'
]
all_cols = set(pl.columns) | set(hl.columns) | set(health.columns) | set(regionl.columns)
variables, values = read_variables(path_values, path_variables)
# filter filtered columns
variables = variables[variables['variable'].isin(all_cols)]
values = values[values['variable'].isin(all_cols)]
# remove duplicates
variables.drop_duplicates(subset='variable', inplace=True)
values.drop_duplicates(subset=['variable', 'value'], inplace=True)

# save the data
variables.to_parquet("./data/variables.parquet.gzip", index=False)
values.to_parquet("./data/values.parquet.gzip", index=False)

print(f"In total {len(variables['variable'])} were selected.")

In total 108 were selected.
